In [ ]:
import feedparser
import dateutil.parser
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import mongo_utils
from datetime import datetime as dt
import locale
from datetime import date, datetime, time
from babel.dates import format_date, format_datetime, format_time


In [ ]:
dbname = mongo_utils.get_database()


In [ ]:
url='https://novini.bg/latest-news'

In [ ]:
collection_name = dbname["novinibg_scrape_1"]
manin_page = requests.get(url)
man_page_data = manin_page.text
main_page = BeautifulSoup(man_page_data, 'html.parser')
for scr_url in main_page.find_all('a', class_="pagination__link -theme-bg-block -theme-border-pagination", href=True): 
    #print(scr_url["href"])
    manin_page = requests.get(scr_url["href"])
    man_page_data = manin_page.text
    main_page = BeautifulSoup(man_page_data, 'html.parser')
    for scr_url2 in main_page.find_all('article', class_="g-grid__item js-content"):   
        for final_url in scr_url2.find_all('a', href=True):
            nlink=final_url["href"]
            #print(nlink)
            if list(collection_name.find({"news_link" : nlink })):
                print("Article Exists")
            else:                
                manin_page = requests.get(nlink)
                man_page_data = manin_page.text
                main_page = BeautifulSoup(man_page_data, 'html.parser')
                news_title=(main_page.find('title').get_text())
                for scr_entr in main_page.find_all('section', class_="openArticle__content --spaced-bottom"):
                    test_lines=scr_entr.find_all('p')
                    art_text=''
                    for line in test_lines:
                        plain_text=line.get_text().replace('  ','').replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', '')
                        art_text += plain_text
                    #print(art_text)
                text=art_text
                dta=main_page.find('p', class_="openArticle__date").get_text().replace('  ','').replace('\n',' ').replace('\xa0', ' ').replace('\r', '').replace('\t', '')
                locale.setlocale(locale.LC_ALL, 'bg_BG.UTF-8')
                date_posted=dt.strptime(dta, '%d %B %Y %H:%M').timestamp()
                news_payload = {
                "date_posted" : date_posted,
                "news_link" : nlink,
                "news_text" : text,
                "news_title" : news_title
                }
                collection_name.insert_many([news_payload])
                print(news_payload)